In [ ]:
###############
### install ###
###############
'''
pip install tweepy
pip install pandas
pip install xlrd
pip install folium
pip install vaderSentiment
pip install tqdm
'''

In [ ]:
########################################
### import the data from Twitter API ###
########################################

import tweepy
import pandas as pd

api_key = 'XXXXX'                                              #put your API keys
api_key_secret = 'XXXXXXX'

access_token = 'XXXXXX'
access_token_secret = 'XXXXX'


auth = tweepy.OAuthHandler(api_key, api_key_secret)             # authentication
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


keywords = '#BTC'                                               # search tweets
new_search = keywords + " -filter:retweets"                     # Exclude retweets in our search
 
date_until = ""                                                 # Define until what date we are looking for tweets ex:"2022-01-01"  
lang = "en"                                                     #if we want to filter by language ex :lang = "en"

limit = 20                                                      #nb of tweets to import

tweets = tweepy.Cursor(api.search_tweets, q=new_search, count=100, tweet_mode='extended',until=date_until,lang=lang).items(limit)   #The above code is using the tweepy library to search for tweets using the search term "new_search"



columns = ['user_name', 'text','date','user_location','user_description','user_followers','user_friends','user_created','user_favourites','source']
data = []                                                       #create DataFrame


for tweet in tweets:                                            #The above code is appending the information of the tweet to the data list
    data.append([tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.user.location, tweet.user.description,tweet.user.followers_count,tweet.user.friends_count,tweet.user.created_at,tweet.user.favourites_count,tweet.source])


df = pd.DataFrame(data, columns=columns)                        #Creating a dataframe with the data and columns we have defined

df.to_csv('data_twitter_api.csv')                               #Saving the dataframe as a csv file

In [ ]:
##############################################################
### cleaning of city data, deletion of duplicate city data ###
##############################################################

import pandas as pd
dfcity = pd.read_csv('worldcities.csv')              #Reading the csv file and storing it in a dataframe called dfcity

i = 0                                                    #i and e are the indexes of the cities in the dataframe dfcity
e = 0
to_drop = []                                             #creation of an empty list

for city in dfcity['city']:
    for city2 in dfcity['city']:
        if city == city2 :                               #Comparing the city names in the dataframe dfcity
            if i != e :                                  #if the index of the 2 citie with the same name is different
                pope = dfcity.population[e]
                popi = dfcity.population[i]
                
                if pope > popi:   
                    to_drop.append(i)                    #for these 2 cities with the same name we add the index of the city with the least population to the list 
                    
                elif popi>pope:
                    to_drop.append(e)                     #for these 2 cities with the same name we add the index of the city with the least population to the list 
                    
        e += 1                                             #add 1 to e to keep the index updated
    i += 1                                                 #add 1 to i to keep the index updated
    e = 0                                                  #reset the index to 0 as the loop starts again



to_drop = list(set(to_drop))                               #many cities in the world have the same name so we delete duplicate elements
to_drop.sort()                                             #It sorts the list in ascending order


for i in (to_drop):
    dfcity=dfcity.drop(i)                                  #Dropping the cities with the same name and the least population.

dfcity.to_csv('data_city_update.csv')                      #Saving the dataframe dfcity in a csv file called data_city_update.csv

In [ ]:
###########################################
### delete tweet lines without location ###
###########################################

import pandas as pd

low_memory=False
df = pd.read_csv('data_twitter_api.csv',engine='python', error_bad_lines=False)   #reading of the two dataset, data with tweet and data of the city
dfcity = pd.read_csv('data_city_update.csv')

try:                                                                               #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    dfcity.pop('Unnamed: 0')                                                       #Because pd.read_csv creates a column 'Unnamed: 0'
except KeyError as err:
        pass

to_fast=len(df)                                                                    #if the data contains too many tweets we can access it by reducing the data by modifying len(df) by the number of data we want
df_fast = df[:to_fast]

i=0                                                                                 #The index of the first element 

while i<len(df_fast):
    try :
        if ',' in df_fast.user_location[i]:                             
            text =df_fast.user_location[i]
            head, sep, tail = text.partition(',')
            df_fast.user_location[i]=head                                          #Removing the comma and after from the user_location column

    except TypeError as err:                                                       #A way to handle the error. If the program encounters a TypeError, it will pass and continue the program
        pass

    i+=1                                                                           #add 1 to e to keep the index updated


e=0                                                                                #The index of the first element 
twi=[]                                                                             #creation of an empty list

for i in df_fast['user_location']:                                                 #loop that will look at each location of user_location in the tweet data set
    
    for a in dfcity['city']:                                                       #another loop that will look at each location of city in the dataset of the world city
        if i == a:                                                                 #check whether the two elements are the same
            twi.append(e)                                                          #if yes, we add e (index element) of the line in data set of tweet
    e+=1                                                                           #add 1 to e to keep the index updated
    
twi = list(set(twi))                                                               #many cities in the world have the same name so we delete duplicate elements
twi.sort()                                                                         #It sorts the list of index of the tweet with the correct location

nb_tweet_in_data = len(twi)-1                                                      #Taking the last index of the list of the index of the tweet with the correct location
df_s = df_fast[:twi[nb_tweet_in_data]]

to_drop = [ele for ele in range(len(df_s)) if ele not in twi]                      #we take the inverse of the elements to know which are the elements without corect localization in order to remove them from the data
to_drop.sort()

for i in (to_drop):                                                                #loop that will drop the line without location in the dataset
    df_s=df_s.drop(i)
    

'''
with open("listTwi.txt", "w") as f:       #Writing the list of the index of the tweet with the correct location in a txt file
    for s in twi:
        f.write(str(s) +",")
'''

df_s.to_csv('data_5K.csv')                                                          #Saving the dataframe df_s to a csv file called data_5K.csv


In [ ]:
##########################################################################
### Adding the latitude and longitude for each location for each tweet ###
##########################################################################

import pandas as pd

df = pd.read_csv('data_5K.csv')                #Reading the csv file
dfcity = pd.read_csv('data_city_update.csv')   #Reading the csv file

try:                                           #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

e=0                                            ##The index of the first element
latitude=[]                                    #creation of an empty list which will be used to indicate the latitude assigned to the city of a tweet
longitude=[]                                   #creation of an empty list which will be used to indicate the longitude assigned to the city of a tweet


for location in df['user_location']:           #This is a loop that is going through the user_location column of the dataframe df
    while location != dfcity.city[e]:          #comparing it to the city column of the dataframe dfcity
        e+=1                                   #add 1 to e to keep the index updated
    latitude.append(dfcity.lat[e])             #if the while loop stops it means that the right city is found, the latitude and longitude of the city is appended to the latitude and longitude list
    longitude.append(dfcity.lng[e])
    


    e=0                                        #reset the index to 0 as the loop starts again
    

df.insert(6,"U_latitude",latitude,True)        #Inserting the latitude and longitude list into the dataframe df at the 6th and 7th column
df.insert(7,"U_longitude",longitude,True)


df.to_csv('data_tweet_with_location.csv')      #Saving the dataframe df into a csv file called tweet_with_location.csv

In [ ]:
#########################################################################
### Converting the date and time from excel format to datetime format ###
#########################################################################

import pandas as pd
import xlrd

df = pd.read_csv('data_tweet_with_location.csv') 

try:                                                            #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

### not useful using Twitter API ###
'''
a=0                                                             #The index of the first element
while a < len(df):                                              #The above code is converting the date and time from excel format to datetime format
    try:
        user_created_date = xlrd.xldate_as_datetime(df.user_created[a], 0)
        df.user_created[a]=user_created_date
        tweet_date = xlrd.xldate_as_datetime(df.date[a], 0)
        df.date[a]=tweet_date
    except TypeError as err:
        pass
    
    a+=1                                                         #add 1 to e to keep the index updated

'''

df.to_csv('data_tweet_with_location_and_date.csv')               #Saving the dataframe df to a csv file

In [ ]:
##################################################
######## extract the hour from the date ##########
### extract the tweet link from the tweet text ###
##################################################

df = pd.read_csv('data_tweet_with_location_and_date.csv') 

try:                                                    #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

hour = []                                               #Creating empty lists to store data 
date = []
tweet = []
link = []

i=0                                                      #The index of the first element
while i < len(df):
    date_hour =df.date[i]
    head, sep, tail = date_hour.partition(' ')          #Splitting the string date_hour into three parts: head wich corespond to the date, sep and tail which corresponds to the hour. The separator is the space ' '
    date.append(head)                                   #appends the date to the date list
    head, sep, tail=tail.partition('+')                 #It splits the string tail into three parts: head wich corespond to the hour, sep and tail which corresponds to the time zone wich is which is incorrect information. The separator is the '+'
    hour.append(head)                                   #It appends the hour to the hour list                         

    tw =df.text[i]                                      #Creating a variable tw which is equal to the text of the tweet at the index i
    
    head, sep, tail = tw.partition('https://t.co/')     #It splits the string tw into three parts: head wich corespond to the text of the tweet, sep and tail which corresponds to the link. The separator is the 'https://t.co/'
    
  
    
    while 'https://t.co/' in tail :                         #Checking if there is stil a link in the tail of the tweet text. If there is this means that there is a link in the tweet before the twitter link of the tweet itself
        lk = tail
        head2, sep, tail = lk.partition('https://t.co/')
    
    if 'https://t.co/' in tail :                            #if there is a link before the Twitter link of the tweet we add head2 to the tweet
        tweet.append(head+head2)                            #It appends the text of the tweet to the tweet list
    else :                                                  #if there is only one link 
        tweet.append(head)                                  #It appends the text of the tweet to the tweet list
    
   
    
    link.append(sep+tail)                               #It appends the link to the link list
    
    i+=1                                                #add 1 to e to keep the index updated



df.pop('date')                                          #Removing the column old 'date'
df.insert(10,"date",date,True)                          #inserting the new columns 'date' and 'hour' from the list at the index 10 and 11
df.insert(11,"hour",hour,True)
df.pop('text')                                          #Removing the column old 'text'
df.insert(12,"tweet",tweet,True)                        #Inserting the new columns 'tweet' and 'link' from the list at the index 12 and 13
df.insert(13,"link",link,True)

df.to_csv('data_tweet_with_location_and_date_hour.csv')  #Saving the dataframe df into a csv file called 'data_tweet_with_location_and_date_hour.csv'

In [ ]:
###########################################
### development of sentimental analysis ###
###########################################

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

df = pd.read_csv('data_tweet_with_location_and_date_hour.csv') 

try:                                                    #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

analyzer = SentimentIntensityAnalyzer()                  #Creating an object of the class SentimentIntensityAnalyzer

sentiment = []                                            #Creating an empty list


for i,s in enumerate(tqdm(df['tweet'],position=0, leave=True)):     #A for loop that iterates over the column 'tweet' 
    vs = analyzer.polarity_scores(str(s))                           #Creating a dictionary with the sentiment of the tweet
    sentiment.append(vs["compound"])                                ##Appending the value of the key 'compound' of the dictionary vs to the list sentiment

df["sentiment"] = sentiment                                 ##Creating a new column in the dataframe df called 'sentiment' and it is filling it with the values of the list sentiment

df.to_csv('data_sentimental.csv')

In [ ]:
#######################################
### creation and display of the map ###
#######################################

import folium
import pandas as pd
import random 
from folium import plugins
from folium import LayerControl
from folium import TileLayer
from folium.plugins import MeasureControl 
from folium.plugins import Fullscreen
from folium.plugins import HeatMap
from folium.plugins import Terminator

df = pd.read_csv('data_sentimental.csv')                        #Reading the csv file and storing it in a dataframe

try:                                                                                  #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

'''create a title'''
loc = 'Tweet about Bitcoin'                                                            #The title of the map
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</h3>'''.format(loc)   

#qf = folium.Figure(width=1000, height=1000)                                            #Creating a figure with a width and height of 1000 pixels
m = folium.Map(location=[0, 0], zoom_start=2, control_scale=True)#.add_to(f)            #Creating a map 

m.get_root().html.add_child(folium.Element(title_html))                                #Adding the title to the map

m.add_child(MeasureControl())                                                          #Adding a measure control to the map

#nbTweet=int(input())                                                                  #Asking the user to input a number
nbTweet=25

displayTweet = random.sample(range(0, len(df)),nbTweet)                               #Creating a list of random numbers between 0 and the number of rows in the dataframe.

color = 'blue'                                                                        #The color of the marker
colorNeg= 'red'
colorPos='green'
'''lightgray', 'darkred', 'blue', 'pink', 'gray', 'red', 'orange', 'lightgreen', 'white', 'beige', 'darkblue', 'green', 'cadetblue', 'darkpurple', 'purple', 'lightblue', 'lightred', 'darkgreen', 'black'''

tooltipNeg = "View Negative tweet!"
tooltipPos = "View Positive tweet!"
tooltip = "View Neutral tweet!"

def popup_html(row):                                                                  #Creating a function that will be used to create the popups
    a = row
    name=df.user_name[a]                                                              #Creating a variable for column in the dataframe
    tweet=df.tweet[a]
    followers=df.user_followers[a]
    date=df.date[a] 
    hour=df.hour[a]                                                                   #The UTC datetime that the List was created on Twitter
    link=df.link[a]

    left_col_color = "#3498DB"                                                        #Setting the color of the left and right column of the table in the popup
    right_col_color = "#E1E8ED"
    
    #CreatinCreating a table with the tweet, link, followers, date and the hour
    #the html code for the popup
    html = """<!DOCTYPE html>
    <html>
    <h5 style="margin-bottom:0";margin-top: 0; width="10px">{}</h5>""".format(name) + """
        <table style="height: 100px; width: 250px;">
    <tbody>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Tweet<br><a href={} target="_blank" style="color:#FFC565;">Link</a></td>""".format(link) + """</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(tweet) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Followers</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(followers) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Date</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(date) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Hour (UTC)</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(hour) + """
    </tr>
    </tbody>
    </table>
    </html>
    """
    return html


i=0
while i < nbTweet :                                                                     #This is a while loop that will run until i is equal to nbTweet
    html = popup_html(displayTweet[i])                                                  #using the html created in the function popup_html
    size= min(max(200,len(str(df.tweet[displayTweet[i]]))*2.2),400)                     #definition of the size of the popup (min = 200, max=400) which takes into account the size of the tweet to display
    iframe = folium.IFrame(html,width=260,height=size)
    popup = folium.Popup(iframe,max_width=300)
    #print(df.sentiment[displayTweet[i]])
    if df.sentiment[displayTweet[i]]< -0.1:
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=colorNeg, icon='twitter', prefix='fa'),tooltip =tooltipNeg).add_to(m)        #creating and adding the marker with popup to the map
    
    elif df.sentiment[displayTweet[i]]> 0.1:
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=colorPos, icon='twitter', prefix='fa'),tooltip =tooltipPos).add_to(m)        #creating and adding the marker with popup to the map
    else :
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=color, icon='twitter', prefix='fa'),tooltip =tooltip).add_to(m)        #creating and adding the marker with popup to the map

    i+=1                                                                                 #add 1 to e to keep the index updated

'''Creating a heat map of the tweets'''
df.U_latitude=df.U_latitude.astype(float)                                                #Converting the latitude and longitude to float
df.U_longitude=df.U_longitude.astype(float)
heat_df=df[["U_latitude","U_longitude"]]                                                 #Creating a dataframe with only the columns U_latitude and U_longitude
heat_data=list(zip(df.U_latitude, df.U_longitude))                                       #Creating a list of tuples with the latitude and longitude
HeatMap(heat_data).add_to(folium.FeatureGroup(name='Heat Map',overlay=True, control=True, show=False).add_to(m))       #Creating and adding the heat map of the tweets to the map

'''Add dark and light mode '''
TileLayer(tiles='cartodbdark_matter',name="dark mode", opacity=0.8,control=True,show=True).add_to(m)     #Adding a dark mode to the map
TileLayer(tiles='cartodbpositron',name="light mode", opacity=0.8,control=True,show=True).add_to(m)
TileLayer(tiles='OpenStreetMap', opacity=0.7,control=True,show=True).add_to(m)

Terminator().add_to(folium.FeatureGroup(name='Day/Night',overlay=True, control=True, show=False).add_to(m))    #shows day and night which can be interesting with very recent tweets

'''Add a layer controller'''
#folium.LayerControl(collapsed=False).add_to(m)
LayerControl(position='topright', collapsed=False, autoZIndex=True, sortLayers=False).add_to(m)          #Adding a layer controller to the map

Fullscreen(position="topright",title="Expand me",title_cancel="Exit me",force_separate_button=True,).add_to(m) #Adding a fullscreen button to the map


'''Add a minimap'''
minimap = plugins.MiniMap()                                                                 #Adding a minimap to the map
m.add_child(minimap)


m.save('map-twiter.html')                                                                   #Saving the map as an html file

m                                                                                           #Display m, the map that we created

